In [32]:
#!pip install pyspark

In [2]:
import pyspark

In [3]:
#Reading Dataset:
import pandas as pd

data = pd.read_csv("weight-height.csv")
data

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801
...,...,...,...
9995,Female,66.172652,136.777454
9996,Female,67.067155,170.867906
9997,Female,63.867992,128.475319
9998,Female,69.034243,163.852461


In [4]:
data.info()     #Its a Pandas DF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gender  10000 non-null  object 
 1   Height  10000 non-null  float64
 2   Weight  10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


#### Starting a Pyspark Session

In [5]:
from pyspark.sql import SparkSession

#Defing Object and Giving spark session a name

spark = SparkSession.builder.appName("Spark-Tutorial").getOrCreate()

In [6]:
spark

#### Reading Dataset in Pyspark `

In [7]:
df_pyspark = spark.read.csv("weight-height.csv")

In [8]:
df_pyspark       #3 columns in my dataset

DataFrame[_c0: string, _c1: string, _c2: string]

In [9]:
df_pyspark.show(5)

+------+----------------+----------------+
|   _c0|             _c1|             _c2|
+------+----------------+----------------+
|Gender|          Height|          Weight|
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|162.310472521300|
|  Male|74.1101053917849|  212.7408555565|
|  Male|71.7309784033377|220.042470303077|
+------+----------------+----------------+
only showing top 5 rows



In [10]:
#Defining Columns Name:
spark.read.csv("weight-height.csv",header=True).show(5)

+------+----------------+----------------+
|Gender|          Height|          Weight|
+------+----------------+----------------+
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|162.310472521300|
|  Male|74.1101053917849|  212.7408555565|
|  Male|71.7309784033377|220.042470303077|
|  Male|69.8817958611153|206.349800623871|
+------+----------------+----------------+
only showing top 5 rows



In [11]:
df_pyspark = spark.read.csv("weight-height.csv",header=True)

##### type of dataset changed from Pandas DF to PYSPARK SQL Database

#### Pyspark DataFrame

In [12]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [13]:
df_pyspark.head(4)

[Row(Gender='Male', Height='73.847017017515', Weight='241.893563180437'),
 Row(Gender='Male', Height='68.7819040458903', Weight='162.310472521300'),
 Row(Gender='Male', Height='74.1101053917849', Weight='212.7408555565'),
 Row(Gender='Male', Height='71.7309784033377', Weight='220.042470303077')]

#### PrintSchema Works like .info

In [14]:
df_pyspark.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)



#### Though Weight and Height are Float, Pyspark has by default taken them to be String!

Pyspark considers all the columns as string until we provide : inferSchema=True

In [15]:
df_pyspark = spark.read.csv("weight-height.csv",header=True,inferSchema=True)
df_pyspark.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)



In [16]:
#Columns Name:

df_pyspark.columns

['Gender', 'Height', 'Weight']

In [17]:
#Selecting Individual Column:

print(df_pyspark["Gender"])   #As a Column 

print(df_pyspark.select("Gender"))    #As a dataframe

Column<'Gender'>
DataFrame[Gender: string]


In [18]:
#Selecting Multiple Column:

print(df_pyspark["Gender","Weight"])   #As a Pyspark Dataframe

print(df_pyspark.select("Gender","Weight"))    #As a dataframe

DataFrame[Gender: string, Weight: double]
DataFrame[Gender: string, Weight: double]


##### Show Applicable to Pyspark DataFrame

In [25]:
df_pyspark["Gender","Weight"].show(3)     

+------+----------------+
|Gender|          Weight|
+------+----------------+
|  Male|241.893563180437|
|  Male|  162.3104725213|
|  Male|  212.7408555565|
+------+----------------+
only showing top 3 rows



In [20]:
# df_pyspark["Gender"].show(3)  #Output is a column hence error

#### Checking Datatypes in Pyspark

In [21]:
df_pyspark.dtypes

[('Gender', 'string'), ('Height', 'double'), ('Weight', 'double')]

In [22]:
df_pyspark.describe().show()

+-------+------+-----------------+------------------+
|summary|Gender|           Height|            Weight|
+-------+------+-----------------+------------------+
|  count| 10000|            10000|             10000|
|   mean|  null|66.36755975482106|161.44035683283076|
| stddev|  null|3.847528120773333|32.108439006519674|
|    min|Female| 54.2631333250971|   64.700126712753|
|    max|  Male| 78.9987423463896|  269.989698505106|
+-------+------+-----------------+------------------+



#### Adding Columns using With Columns:

In [23]:
df_pyspark= df_pyspark.withColumn("Weight After 2 Years",df_pyspark["Weight"]+5)

In [24]:
df_pyspark.describe().show()

+-------+------+-----------------+------------------+--------------------+
|summary|Gender|           Height|            Weight|Weight After 2 Years|
+-------+------+-----------------+------------------+--------------------+
|  count| 10000|            10000|             10000|               10000|
|   mean|  null|66.36755975482106|161.44035683283076|   166.4403568328308|
| stddev|  null|3.847528120773333|32.108439006519674|  32.108439006519674|
|    min|Female| 54.2631333250971|   64.700126712753|     69.700126712753|
|    max|  Male| 78.9987423463896|  269.989698505106|    274.989698505106|
+-------+------+-----------------+------------------+--------------------+



#### Dropping Columns 

In [26]:
df_pyspark= df_pyspark.drop("Weight After 2 Years")

In [29]:
df_pyspark.show(3)

+------+----------------+----------------+
|Gender|          Height|          Weight|
+------+----------------+----------------+
|  Male| 73.847017017515|241.893563180437|
|  Male|68.7819040458903|  162.3104725213|
|  Male|74.1101053917849|  212.7408555565|
+------+----------------+----------------+
only showing top 3 rows



#### Renaming Column:

In [31]:
df_pyspark = df_pyspark.withColumnRenamed("Gender","Sex")
df_pyspark.show(2)

+----+----------------+----------------+
| Sex|          Height|          Weight|
+----+----------------+----------------+
|Male| 73.847017017515|241.893563180437|
|Male|68.7819040458903|  162.3104725213|
+----+----------------+----------------+
only showing top 2 rows



#### Dealing with Null values

In [41]:
df_pyspark.na.drop()           #Dropping all null values

DataFrame[Sex: string, Height: double, Weight: double]

In [42]:
#Using ANY and ALL in HOW
df_pyspark.na.drop(how="all")         #Delete only those rows which have all values null 

df_pyspark.na.drop(how="any")         #Delete  rows which have even 1 value as null 

DataFrame[Sex: string, Height: double, Weight: double]

In [ ]:
#Using Threshold

df_pyspark.na.drop(how="any",thresh=2)     #At-least 2 non-null values should be present, if less than 2 values delete row

In [ ]:
#Using Subset

df_pyspark.na.drop(how="any",subset=["Weight"])    #Delete all those rows in which Weight Values are Null


In [ ]:
#Filling Missing Values

df_pyspark.na.fill("NA")

df_pyspark.na.fill("NA","Weight")    #Filling Missing values in Weight column with NA

df_pyspark.na.fill("NA","Weight")

#### Using Imputer to Fill Missing Values

In [52]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=["Weight","Height"],
                  outputCols=["{} imputed".format(c) for c in ["Weight","Height"]]).setStrategy("mean")

imputer.fit(df_pyspark).transform(df_pyspark).show(4)                 #Replaced the missing values with mean

#Can be similarly done for median , mode etc

+----+----------------+----------------+----------------+----------------+
| Sex|          Height|          Weight|  Weight imputed|  Height imputed|
+----+----------------+----------------+----------------+----------------+
|Male| 73.847017017515|241.893563180437|241.893563180437| 73.847017017515|
|Male|68.7819040458903|  162.3104725213|  162.3104725213|68.7819040458903|
|Male|74.1101053917849|  212.7408555565|  212.7408555565|74.1101053917849|
|Male|71.7309784033377|220.042470303077|220.042470303077|71.7309784033377|
+----+----------------+----------------+----------------+----------------+
only showing top 4 rows



#### Filter Operations

In [58]:
#Weight less than 170
print(df_pyspark.count())
df_pyspark.filter("Weight<=170").count()

10000


5797

- row_number = data.count()
- column_number = len(data.dtypes)
- Shape : print((df.count(), len(df.columns)))

In [64]:
print(len(df_pyspark.columns))

df_pyspark.count(),len(df_pyspark.columns)

3


(10000, 3)

### Filtering and Selecting

In [67]:
df_pyspark.filter("Weight<=170").select(["Weight","Height"]).show(6)

+----------------+----------------+
|          Weight|          Height|
+----------------+----------------+
|  162.3104725213|68.7819040458903|
|152.212155757083|67.2530156878065|
|167.971110489509|68.3485155115879|
|156.399676387112|63.4564939783664|
|167.127461073476|64.7663291334055|
|149.173566007975|66.1491319608781|
+----------------+----------------+
only showing top 6 rows



#### AND

In [72]:
df_pyspark.filter("Weight<=170" and "Height>70").select(["Weight","Height"]).show(1)

+----------------+---------------+
|          Weight|         Height|
+----------------+---------------+
|241.893563180437|73.847017017515|
+----------------+---------------+
only showing top 1 row



In [74]:
df_pyspark.filter((df_pyspark["Weight"]<=170) & (df_pyspark["Height"]>70)).select(["Weight","Height"]).show(1)

+----------------+----------------+
|          Weight|          Height|
+----------------+----------------+
|167.301408606857|70.0941169135012|
+----------------+----------------+
only showing top 1 row



#### OR

In [71]:
df_pyspark.filter("Weight<=170" or "Height>70").select(["Weight","Height"]).show(6)

+----------------+----------------+
|          Weight|          Height|
+----------------+----------------+
|  162.3104725213|68.7819040458903|
|152.212155757083|67.2530156878065|
|167.971110489509|68.3485155115879|
|156.399676387112|63.4564939783664|
|167.127461073476|64.7663291334055|
|149.173566007975|66.1491319608781|
+----------------+----------------+
only showing top 6 rows



#### NOT

In [86]:
df_pyspark.filter(~(df_pyspark["Weight"]<=170)).select(["Weight","Height"]).show(6)

+----------------+----------------+
|          Weight|          Height|
+----------------+----------------+
|241.893563180437| 73.847017017515|
|  212.7408555565|74.1101053917849|
|220.042470303077|71.7309784033377|
|206.349800623871|69.8817958611153|
|183.927888604031|68.7850812516616|
| 175.92944039571| 67.018949662883|
+----------------+----------------+
only showing top 6 rows



#### PySpark GroupBy and Aggregate:

In [88]:
#Groupby sum

df_pyspark.groupby("Sex").sum().show()

+------+------------------+-----------------+
|   Sex|       sum(Height)|      sum(Weight)|
+------+------------------+-----------------+
|Female| 318543.8680171246|679300.4650373434|
|  Male|345131.72953108686|935103.1032909645|
+------+------------------+-----------------+



In [89]:
#Groupby count 
df_pyspark.groupby("Sex").count().show()

+------+-----+
|   Sex|count|
+------+-----+
|Female| 5000|
|  Male| 5000|
+------+-----+



In [90]:
#Groupby Max
df_pyspark.groupby("Sex").max().show()

+------+----------------+----------------+
|   Sex|     max(Height)|     max(Weight)|
+------+----------------+----------------+
|Female|73.3895858660697|202.237213739559|
|  Male|78.9987423463896|269.989698505106|
+------+----------------+----------------+



In [91]:
#Groupby Min
df_pyspark.groupby("Sex").min().show()

+------+----------------+----------------+
|   Sex|     min(Height)|     min(Weight)|
+------+----------------+----------------+
|Female|54.2631333250971| 64.700126712753|
|  Male|58.4069049317498|112.902939447818|
+------+----------------+----------------+



In [95]:
#Groupby Avg
df_pyspark.groupby("Sex").avg().show()

+------+------------------+-----------------+
|   Sex|       avg(Height)|      avg(Weight)|
+------+------------------+-----------------+
|Female|63.708773603424916|135.8600930074687|
|  Male| 69.02634590621737|187.0206206581929|
+------+------------------+-----------------+



In [97]:
#Groupby mean
df_pyspark.groupby("Sex").mean().show()

+------+------------------+-----------------+
|   Sex|       avg(Height)|      avg(Weight)|
+------+------------------+-----------------+
|Female|63.708773603424916|135.8600930074687|
|  Male| 69.02634590621737|187.0206206581929|
+------+------------------+-----------------+



In [99]:
#Aggregate

#Overall sum of Height
df_pyspark.agg({"Height":"sum"}).collect()

[Row(sum(Height)=663675.5975482106)]

In [101]:
df_pyspark.agg({"Height":"max"}).show()    #Directly applying aggregate functions

+----------------+
|     max(Height)|
+----------------+
|78.9987423463896|
+----------------+



##### Types of Aggregate functions: Sum, Count, Avg etc : Applied after grouping

In [104]:
df_pyspark.groupby("Sex").agg({"Height":"max"}).show()

+------+----------------+
|   Sex|     max(Height)|
+------+----------------+
|Female|73.3895858660697|
|  Male|78.9987423463896|
+------+----------------+

